## Notebook Overview

This notebook builds upon the feature-engineered dataset from the previous notebook (`04_feature_engineering.ipynb`) and focuses on **Model Training and Evaluation**. Our primary goal is to develop a credit risk prediction model that excels at identifying potential loan defaulters, thereby minimizing financial losses for retail banks while also considering their desired balance between risk aversion and loan approval rates. This translates to maximizing the recall of the positive class (loan defaulters) while maintaining acceptable precision and overall model performance.

### 0.5.1 Objectives

The main objectives of this notebook are:

1. **Model Selection:** Choose algorithms suitable for imbalanced classification problems.
2. **Model Training:** Train models with a focus on identifying potential defaulters.
3. **Hyperparameter Tuning:** Optimize models to increase recall for the positive class.
4. **Model Evaluation:** Assess models primarily on recall, while considering precision, F2-score, AUC-PR, and overall performance.
5. **Model Comparison:** Compare different models based on their ability to identify true positives and balance the precision-recall trade-off.
6. **Threshold Adjustment:** Explore the impact of classification thresholds on recall and precision, collaborating with retail banks to determine the optimal threshold.

### 0.5.2 Importance of Focusing on Recall

Prioritizing recall for defaulter prediction is crucial for minimizing financial losses, which is the primary business objective in credit risk assessment. The cost of missing a potential defaulter (false negative) is typically much higher than the cost of incorrectly classifying a non-defaulter as high-risk (false positive). While we prioritize recall, we will also carefully consider the precision-recall trade-off and aim for a model that maximizes recall without severely impacting precision. Techniques like threshold adjustment and cost-sensitive learning will be used to balance these metrics effectively. Furthermore, demonstrating a thorough approach to risk identification aligns with regulatory expectations in the financial sector, supporting the banks' compliance needs. This approach also allows for more conservative lending practices, which can be adjusted based on the bank's specific risk tolerance.

### 0.5.3 Our Approach

In this notebook, we will focus on the following modeling tasks:

1. **Data Preparation:** Address class imbalance using techniques like SMOTE or class weighting.
2. **Baseline Model:** A logistic regression model with class weights inversely proportional to class frequencies will serve as our baseline. This will provide a benchmark for evaluating more complex models.
3. **Advanced Models:** Train and evaluate models known for handling imbalanced data:
   - Decision Trees with adjusted class weights
   - Random Forest with balanced class weights
   - Gradient Boosting (XGBoost, LightGBM) with `scale_pos_weight` adjustment
4. **Hyperparameter Tuning:** We will employ techniques like GridSearchCV or RandomizedSearchCV, optimizing for the F2-score (which gives more weight to recall) or a custom cost-sensitive scoring function.
5. **Model Evaluation:** Prioritize recall in our metrics, while also considering precision, F2-score, AUC-PR, and AUC-ROC.
6. **Threshold Adjustment:** We will experiment with different classification thresholds and work closely with retail banks to determine the optimal threshold that balances their desired level of risk aversion with acceptable loan approval rates.
7. **Ensemble Methods:** Explore ensemble techniques that can improve recall without severely impacting precision.
8. **Cost-Sensitive Learning:** Incorporate misclassification costs to reflect the higher cost of false negatives, aligning the model's objective with the business goal of minimizing financial losses.

By the end of this notebook, we aim to have a model (or ensemble of models) that excels at identifying potential loan defaulters, providing the bank with a powerful tool for risk assessment and mitigation.


In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import (
    train_test_split,
    StratifiedKFold,
    GridSearchCV,
    RandomizedSearchCV,
    cross_val_score,
)
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
import xgboost as xgb
from sklearn.metrics import (
    classification_report,
    roc_auc_score,
    precision_recall_curve,
    auc,
    confusion_matrix,
    ConfusionMatrixDisplay,
    f1_score
)

from retail_bank_risk.model_training_utils import downscale_dtypes

from sklearn.metrics import make_scorer, recall_score
from joblib import Parallel, delayed
import warnings
warnings.filterwarnings('ignore')


In [2]:
train_df = pd.read_parquet(
    "../data/processed/application_train_engineered.parquet"
)
test_df = pd.read_parquet(
    "../data/processed/application_test_engineered.parquet"
)

In [3]:
print(f"Training Data Shape: {train_df.shape}")
print(f"Test Data Shape: {test_df.shape}")

train_df.head()

Training Data Shape: (307511, 77)
Test Data Shape: (48744, 76)


,reg_city_not_work_city_0,reg_city_not_work_city_1,region_rating_client_w_city,region_rating_client,name_contract_type_cash loans,name_contract_type_revolving loans,code_gender_m,code_gender_f,flag_own_car_n,flag_own_car_y,...,is_anomaly_true,age_group,income_group,credit_amount_group,debt_to_income_ratio,credit_to_goods_ratio,annuity_to_income_ratio,ext_source_mean,credit_exceeds_goods,target
0,1,0,1.0,1.0,1,0,1,0,1,0,...,0,1.0,3.0,1.0,2.007889,1.158397,0.121978,0.201162,1,1
1,1,0,2.0,2.0,1,0,0,1,1,0,...,0,3.0,4.0,4.0,4.790750,1.145199,0.132217,0.588812,1,0
2,1,0,1.0,1.0,0,1,1,0,0,1,...,0,3.0,0.0,0.0,2.000000,1.000000,0.100000,0.642739,0,0
3,1,0,1.0,1.0,1,0,0,1,1,0,...,0,3.0,1.0,1.0,2.316167,1.052803,0.219900,0.680460,1,0
4,0,1,1.0,1.0,1,0,1,0,1,0,...,0,3.0,1.0,2.0,4.222222,1.000000,0.179963,0.397760,0,0


Seems everything fine, encoded variables are saved, new features created, shapes align with the expected setup.

Next, we can downscale feature dtypes for last time.

In [4]:
train_df, test_df = downscale_dtypes(train_df, test_df)

train_df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307511 entries, 0 to 307510
Data columns (total 77 columns):
 #   Column                                   Non-Null Count   Dtype   
---  ------                                   --------------   -----   
 0   reg_city_not_work_city_0                 307511 non-null  uint8   
 1   reg_city_not_work_city_1                 307511 non-null  uint8   
 2   region_rating_client_w_city              307511 non-null  float32 
 3   region_rating_client                     307511 non-null  float32 
 4   name_contract_type_cash loans            307511 non-null  uint8   
 5   name_contract_type_revolving loans       307511 non-null  uint8   
 6   code_gender_m                            307511 non-null  uint8   
 7   code_gender_f                            307511 non-null  uint8   
 8   flag_own_car_n                           307511 non-null  uint8   
 9   flag_own_car_y                           307511 non-null  uint8   
 10  flag_own_realty_y   

In [5]:
X = train_df.drop("target", axis=1)
y = train_df["target"]

print(f"Number of Features: {X.shape[1]}")
print("Feature Names:", X.columns.tolist())

Number of Features: 76
Feature Names: ['reg_city_not_work_city_0', 'reg_city_not_work_city_1', 'region_rating_client_w_city', 'region_rating_client', 'name_contract_type_cash loans', 'name_contract_type_revolving loans', 'code_gender_m', 'code_gender_f', 'flag_own_car_n', 'flag_own_car_y', 'flag_own_realty_y', 'flag_own_realty_n', 'name_type_suite_unaccompanied', 'name_type_suite_family', 'name_type_suite_spouse, partner', 'name_type_suite_children', 'name_type_suite_other_a', 'name_type_suite_mode', 'name_type_suite_other_b', 'name_type_suite_group of people', 'name_income_type_working', 'name_income_type_state servant', 'name_income_type_commercial associate', 'name_income_type_pensioner', 'name_income_type_unemployed', 'name_income_type_student', 'name_income_type_businessman', 'name_income_type_maternity leave', 'name_education_type', 'name_family_status_single / not married', 'name_family_status_married', 'name_family_status_civil marriage', 'name_family_status_widow', 'name_famil

In [6]:
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Training Set Shape: {X_train.shape}")
print(f"Validation Set Shape: {X_val.shape}")

Training Set Shape: (246008, 76)
Validation Set Shape: (61503, 76)


Prioritize recall to identify as many defaulters as possible.

In [7]:
recall_scorer = make_scorer(recall_score, pos_label=1, zero_division=0)

Define pipelines for different models.

This way, we ensure the same preprocessing steps are applied within each model's evaluation during cross-validation, preventing data leakage and making results more reliable.


In [8]:
pipeline_lr = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', LogisticRegression(random_state=42, max_iter=1000))
])

# Decision Tree Pipeline
pipeline_dt = Pipeline([
    ('classifier', DecisionTreeClassifier(random_state=42))
])

# Random Forest Pipeline
pipeline_rf = Pipeline([
    ('classifier', RandomForestClassifier(random_state=42, n_jobs=-1))
])

# XGBoost Pipeline
pipeline_xgb = Pipeline([
    ('classifier', xgb.XGBClassifier(
        objective="binary:logistic",
        use_label_encoder=False,
        eval_metric='logloss',
        random_state=42,
        n_jobs=-1
    ))
])

Define parameter grids for each model


In [9]:
# Logistic Regression Parameter Grid
param_grid_lr = {
    'classifier__C': [0.001, 0.01, 0.1, 1, 10, 100],
    'classifier__class_weight': ['balanced', {0:1, 1:2}, {0:1, 1:5}, {0:1, 1:10}],
    'classifier__penalty': ['l1', 'l2'],  # Removed 'elasticnet' and 'none'
    'classifier__solver': ['liblinear', 'saga']  # Solvers compatible with 'l1' and 'l2'
}

# Decision Tree Parameter Grid
param_grid_dt = {
    'classifier__max_depth': [None, 10, 20, 30],
    'classifier__min_samples_split': [2, 5, 10],
    'classifier__min_samples_leaf': [1, 2, 4],
    'classifier__class_weight': ['balanced', {0:1, 1:2}, {0:1, 1:5}, {0:1, 1:10}]
}

# Random Forest Parameter Grid
param_grid_rf = {
    'classifier__n_estimators': [50, 100, 200],
    'classifier__max_depth': [None, 10, 20, 30],
    'classifier__min_samples_split': [2, 5, 10],
    'classifier__min_samples_leaf': [1, 2, 4],
    'classifier__class_weight': ['balanced', {0:1, 1:2}, {0:1, 1:5}, {0:1, 1:10}]
}

# XGBoost Parameter Grid
param_grid_xgb = {
    'classifier__n_estimators': [50, 100, 200],
    'classifier__learning_rate': [0.01, 0.1, 0.3],
    'classifier__max_depth': [3, 5, 7],
    'classifier__scale_pos_weight': [1, 2, 5, 10]
}

In [10]:
pipelines = [pipeline_lr, pipeline_dt, pipeline_rf, pipeline_xgb]
param_grids = [param_grid_lr, param_grid_dt, param_grid_rf, param_grid_xgb]
model_names = ['Logistic Regression', 'Decision Tree', 'Random Forest', 'XGBoost']
best_estimators = []

# Define Stratified K-Fold Cross-Validation
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Perform Grid Search for each model with parallel processing
for name, pipeline, param_grid in zip(model_names, pipelines, param_grids):
    print(f"Starting Grid Search for {name}...")
    grid_search = GridSearchCV(
        pipeline,
        param_grid=param_grid,
        cv=kfold,
        scoring=recall_scorer,
        n_jobs=-1,  # Utilize all available cores
        verbose=2,
        error_score='raise'  # Raise errors to debug
    )
    try:
        grid_search.fit(X_train, y_train)
        best_estimators.append(grid_search.best_estimator_)
        print(f"Best parameters for {name}: {grid_search.best_params_}")
        print(f"Best recall score: {grid_search.best_score_:.4f}\n")
    except Exception as e:
        print(f"Grid Search failed for {name} with error: {e}\n")

Starting Grid Search for Logistic Regression...
Fitting 5 folds for each of 96 candidates, totalling 480 fits
Grid Search failed for Logistic Regression with error: pos_label=1 is not a valid label: It should be one of ['0' '1']

Starting Grid Search for Decision Tree...
Fitting 5 folds for each of 144 candidates, totalling 720 fits
Grid Search failed for Decision Tree with error: pos_label=1 is not a valid label: It should be one of ['0' '1']

Starting Grid Search for Random Forest...
Fitting 5 folds for each of 432 candidates, totalling 2160 fits
Grid Search failed for Random Forest with error: pos_label=1 is not a valid label: It should be one of ['0' '1']

Starting Grid Search for XGBoost...
Fitting 5 folds for each of 108 candidates, totalling 540 fits
Grid Search failed for XGBoost with error: Invalid classes inferred from unique values of `y`.  Expected: [0 1], got ['0' '1']



In [13]:
for name, model in zip(model_names, best_estimators):
    print(f"Evaluating {name}...")

    # Cross-validated recall
    cv_results = cross_val_score(
        model,
        X_train,
        y_train,
        cv=kfold,
        scoring=recall_scorer,
        n_jobs=-1  # Utilize all cores
    )
    print(f"{name} Cross-Validated Recall: {cv_results.mean():.4f} (± {cv_results.std():.4f})")

    # Predictions on Validation Set
    y_pred = model.predict(X_val)
    y_pred_proba = model.predict_proba(X_val)[:, 1]

    # Classification Report
    print("Classification Report:")
    print(classification_report(y_val, y_pred))

    # Precision-Recall AUC
    precision, recall, thresholds = precision_recall_curve(y_val, y_pred_proba)
    pr_auc = auc(recall, precision)
    print(f"AUC-PR: {pr_auc:.4f}")

    # ROC AUC
    roc_auc = roc_auc_score(y_val, y_pred_proba)
    print(f"AUC-ROC: {roc_auc:.4f}")

    # F1 Score
    f1 = f1_score(y_val, y_pred)
    print(f"F1 Score: {f1:.4f}")

    # Confusion Matrix
    cm = confusion_matrix(y_val, y_pred)

    print("\n" + "="*60 + "\n")

In [12]:
train_df.shape

(307511, 77)